# Mockup Version 1

- Mock-up the workflow from MIS global summit in an environment like EvidenceCare's Epic App. Due January 1.
    - ☑️ List content for our app (Due by Nov 23) TJM
    - ☑️ Define the workflow between user, website, and service (Due by Nov 23) TJM & NS workshop session
    - ☑️ Define interface between UI and service (Due by Nov 30) TJM & NS workshop session
    - 🟢 Include procedures predictor (Due by Dec 14) TJM
    - 🟢 Write a dummy function that returns assessment and plan in JSON format (Due by Dec 14) TJM
    - ☑️ Create a static page layout (Due by Dec 7) NS
    - ☑️ Flask configuration (due by Dec 1) TJM 
    - Tim send Nikesh screenshot of Epic data
    - Nikesh will send code to incorporate into fetchData and app.y 
    - Write & test the code for webpage (Due by Dec 23) TJM & NS
    - Provide instructions, documentation, supporting information to make it (Due by Dec 23) BED

In [3]:
import pandas as pd
import time

In [4]:
# import getpass
# password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")

from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j','NikeshIsCool'))
session=driver.session()

In [18]:
# get a CSV of all problems in the database that have enough data to potentially return robust results
query = '''
MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
WITH round(r.co_occurrance_probability, 5)*1000 AS Score, ord, r, c
WHERE Score > 20
RETURN distinct(c.term) AS Problem, c.cui'''
data = session.run(query)

# Convert the neo4j object into a dataframe
common_problems_df = pd.DataFrame([dict(record) for record in data])

In [21]:
common_problems_df.sort_values(by='Problem', inplace=True)
common_problems_df.Problem.to_list()

['Abnormally opaque structure (morphologic abnormality)',
 'Acidosis',
 'Acidosis, Lactic',
 'Acidosis, Respiratory',
 'Actinic cheilitis',
 'Acute respiratory failure',
 'Aggressive Periodontitis',
 'Agitation',
 'Alcohol Withdrawal Delirium',
 'Alcohol abuse',
 'Alcohol withdrawal syndrome',
 'Alkalosis, Respiratory',
 'Aneurysm',
 'Apnea',
 'Ascites',
 'Back Pain',
 'Blood Clot',
 'Bronchospasm',
 'Cardiac Arrhythmia',
 'Cerebrovascular accident',
 'Chest Pain',
 'Cholangiocarcinoma',
 'Chronic Kidney Diseases',
 'Chronic back pain',
 'Confusion',
 'Contraction (finding)',
 'Coughing',
 'Dentatorubral-Pallidoluysian Atrophy',
 'Desmoplastic infantile ganglioglioma',
 'Diabetes Mellitus, Non-Insulin-Dependent',
 'Diabetic Ketoacidosis',
 'Diastolic dysfunction',
 'Disease',
 'Disease Progression',
 'Dry skin',
 'Dyspnea',
 'Encephalitis',
 'Epilepsy',
 'Esophageal Varices',
 'Fatigue',
 'Febrile Neutropenia',
 'Flare',
 'Fluid overload',
 'Gastrointestinal Hemorrhage',
 'Gastroparesi

In [26]:
# Trim out problems that seem to be spurious
spurious = ['Abnormally opaque structure (morphologic abnormality)', 'Actinic cheilitis', 'Aggressive Periodontitis', 'Dentatorubral-Pallidoluysian Atrophy', 'Desmoplastic infantile ganglioglioma',  'Disease', 'Disease Progression', 'Flare', 'Obstruction', 'POPLITEAL PTERYGIUM SYNDROME, LETHAL TYPE',  'Pathological Dilatation', 'Pathology processes', 'Sore to touch', 'Xanthomatosis, Cerebrotendinous',  'diastolic congestive heart failure', 'gastrointestinal gas', 'physiologic resolution']
common_problems_df = common_problems_df[~common_problems_df.Problem.isin(spurious)]

In [27]:
# Write out to CSV
common_problems_df.to_csv('common_problems.csv', index=False, encoding='utf-8')

## Modify existing functions for the new data model
### Input a list of CUIs for known problems, output possible additional problems with their CUIs and odds ratios in a dataframe
Note: This query relies on the OCCURS_WITH relationship, which was created in the [Virtual_Population](Documents/P2_Prototype/Virtual_Population.ipynb) notebook under the section "Using Conditional Probability instead of Odds Ratio."

In [33]:
def PotentialComorbidities(cui_prob_list):
    
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Pathologic Function']", "['Disease or Syndrome']", "['Mental or Behavioral Dysfunction']", "['Sign or Symptom']", "['Injury or Poisoning']", "['Neoplastic Process']"]
    WITH round(r.co_occurrance_probability, 5)*1000 AS Score, ord, r
    WHERE Score > 20
    RETURN ord.cui AS `CUI`, ord.term AS `Problem`, Score
    ORDER BY r.co_occurrance_probability DESC
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    likely_comorbidities = pd.DataFrame([dict(record) for record in data])
    
    return likely_comorbidities

In [34]:
# Test the function
start_time = time.time()

cui_prob_list = ['C1565489', 'C0085762']
PotentialComorbidities = PotentialComorbidities(cui_prob_list)

print("Total runtime:", time.time() - start_time, "seconds")
PotentialComorbidities

Total runtime: 0.049037933349609375 seconds


,CUI,Problem,Score
0,C0220982,Ketoacidosis,155.34
1,C0149521,"Pancreatitis, Chronic",143.31
2,C0151723,Hypomagnesemia,130.16
3,C0236663,Alcohol withdrawal syndrome,128.93
4,C1963823,Hyperlipasaemia,118.59
5,C0438696,Suicidal,116.84
6,C0747752,Polysubstance abuse,115.94
7,C0162316,Iron deficiency anemia,111.78
8,C3159311,BORNHOLM EYE DISEASE,108.70
9,C0232498,Abdominal tenderness,107.26


### Input 1 problem, output the most likely orders
Note: This query relies on the OCCURS_WITH relationship, which was created in the [Virtual_Population](Documents/P2_Prototype/Virtual_Population.ipynb) notebook under the section "Using Conditional Probability instead of Odds Ratio."

In [23]:
def LikelyOrders(cui_prob_list):
    
    # Find prescriptions associated with the input problem    
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Clinical Drug']"]
    WITH round(r.co_occurrance_probability, 5)*1000 AS Score, ord, r
    WHERE Score > 20
    RETURN ord.cui AS `Code`, ord.term AS `Order`, Score
    ORDER BY r.co_occurrance_probability DESC
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    orders_likely_rx = pd.DataFrame([dict(record) for record in data]).head(10)
    
    # Find abnormal labs associated with the input problem
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Clinical Attribute']"]
    WITH round(r.co_occurrance_probability, 5)*1000 AS Score, ord, r
    WHERE Score > 20
    RETURN ord.cui AS `Code`, ord.description AS `Order`, Score
    ORDER BY r.co_occurrance_probability DESC
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    orders_likely_lab = pd.DataFrame([dict(record) for record in data]).head(10)
    
    # Find procedures associated with the input problem
    query = '''
    MATCH p=(ord:D_Icd_Procedures)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list}
    WITH round(r.co_occurrance_probability, 5)*1000 AS Score, ord, r
    WHERE Score > 20
    RETURN ord.icd9_code AS `Code`, ord.long_title AS `Order`, Score
    ORDER BY r.co_occurrance_probability DESC
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    orders_likely_procedure = pd.DataFrame([dict(record) for record in data]).head(10)
    
    return orders_likely_rx, orders_likely_lab, orders_likely_procedure

In [ ]:
MATCH (source:Concept {cui:'C0022661'})
MATCH (destination:Concept {cui:'C0975120'})
CALL gds.alpha.shortestPath.stream({
    startNode: source,
    endNode: destination,
    nodeProjection: 
})

In [24]:
start_time = time.time()

cui_prob_list = ['C0022661']
orders_likely_rx, orders_likely_lab, orders_likely_procedure = LikelyOrders(cui_prob_list)

print("Total runtime:", time.time() - start_time, "seconds")

Total runtime: 0.014989852905273438 seconds


In [25]:
orders_likely_rx

,Code,Order,Score
0,C0975120,calcitriol 0.00025 MG Oral Capsule,109.59
1,C1967412,sevelamer carbonate 800 MG Oral Tablet [Renvela],106.88
2,C1951501,sodium polystyrene sulfonate 250 MG/ML Oral Su...,103.32
3,C0706911,1 ML epoetin alfa 4000 UNT/ML Injection [Procrit],79.31
4,C4082775,150 ML Glucose 50 MG/ML Injection,76.26
5,C0710995,purified protein derivative of tuberculin 50 U...,70.99
6,C0691839,sodium bicarbonate 650 MG Oral Tablet,66.74
7,C0978650,midodrine hydrochloride 5 MG Oral Tablet,64.02
8,C1169852,piperacillin 2000 MG / tazobactam 250 MG Injec...,62.86
9,C0975350,cefepime 1000 MG Injection,60.51


In [26]:
orders_likely_lab

,Code,Order,Score
0,C1544491,Macrophage in Ascites,57.87
1,C0549816,Lymphocytes in Other Body Fluid,57.69
2,C0942476,Polys in Other Body Fluid,53.77
3,C0942444,Lymphocytes in Ascites,52.82
4,C0942425,"RBC, Ascites in Ascites",51.48
5,C0945358,"WBC, Ascites in Ascites",50.71
6,C0942450,Monocytes in Ascites,49.68
7,C0942478,Polys in Ascites,48.77
8,C0365034,Protein/Creatinine Ratio in Urine,48.01
9,C0368042,pH in Urine,44.46


In [27]:
orders_likely_procedure

,Code,Order,Score
0,3995,Hemodialysis,69.13
1,3895,Venous catheterization for renal dialysis,64.32
2,4513,Other endoscopy of small intestine,24.70
3,3324,Closed [endoscopic] biopsy of bronchus,23.03
4,3893,"Venous catheterization, not elsewhere classified",21.32
5,9904,Transfusion of packed cells,21.29
6,9604,Insertion of endotracheal tube,20.16
